In [1]:
import numpy as np # linear algebra
import pandas as pd # dataframes
import matplotlib.pyplot as plt # General visualisations
import matplotlib.ticker as mtick # Axis visuals
import seaborn as sns # Statistical visualisations
from math import pi # Radar chart support
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, LabelEncoder, RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report 

from sklearn import svm

In [56]:
from sklearn.model_selection import StratifiedKFold, KFold
def cross_validation(X,y,model):
    f1 = []
    conv = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1_macro = f1_score(y_test, y_pred, average = 'macro')
       
        f1.append(f1_macro)
        #conv.append(confusion_matrix(y_test, y_pred))
        print(f'here is the : {f1_macro}')
        print(classification_report(y_test, y_pred))
        print("==="*20)
    return f1, conv

In [57]:
train = pd.read_csv('../data/preprocess/train_2.csv')
train = train.iloc[:,2:]

/var/folders/yt/n9_2kr5s6nx_5b2y_kx3mnnm0000gn/T/ipykernel_39890/592460438.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/preprocess/train_2.csv')


In [58]:
train_data_new = train.dropna(subset = ['technical_solution_proposed'])
train_data_new.isna().sum()

floors_before_eq (total)          0
old_building                      0
plinth_area (ft^2)                0
height_before_eq (ft)             0
land_surface_condition            0
type_of_foundation                0
type_of_roof                      0
type_of_ground_floor              0
type_of_other_floor               0
position                          0
building_plan_configuration       0
technical_solution_proposed       0
legal_ownership_status            0
has_secondary_use                 0
type_of_reinforcement_concrete    0
residential_type                  0
no_family_residing                0
public_place_type                 0
industrial_use_type               0
govermental_use_type              0
flexible_superstructure           0
wall_binding                      0
wall_material                     0
damage_grade                      0
dtype: int64

In [59]:
obj_cols = train_data_new.select_dtypes(include='object').columns.tolist()
num_cols = [i for i in train.columns[:-1] if i not in obj_cols]
tmp_train = train_data_new.copy()

In [60]:
num_cols

['floors_before_eq (total)',
 'old_building',
 'plinth_area (ft^2)',
 'height_before_eq (ft)',
 'has_secondary_use',
 'type_of_reinforcement_concrete',
 'no_family_residing',
 'wall_binding',
 'wall_material']

In [61]:
### binding 
for i in num_cols[:5]:
    for c in num_cols:
        tmp_train[f'{c}_nominal'] = pd.cut(tmp_train[c], bins=5, labels=[1,2,3,4,5])

In [62]:
obj_cols

['land_surface_condition',
 'type_of_foundation',
 'type_of_roof',
 'type_of_ground_floor',
 'type_of_other_floor',
 'position',
 'building_plan_configuration',
 'technical_solution_proposed',
 'legal_ownership_status',
 'residential_type',
 'public_place_type',
 'industrial_use_type',
 'govermental_use_type',
 'flexible_superstructure']

In [63]:
### onehot
tmp_var_ohe = pd.get_dummies(tmp_train[obj_cols])
tmp_var_ohe

,land_surface_condition_Flat,land_surface_condition_Moderate slope,land_surface_condition_Steep slope,type_of_foundation_bamboo/timber,type_of_foundation_cement-stone/brick,type_of_foundation_clay mortar-stone/brick,type_of_foundation_mud mortar-stone/brick,type_of_foundation_other,type_of_foundation_rc,type_of_roof_bamboo/timber heavy roof,...,industrial_use_type_Infrastructure,industrial_use_type_Manufacturing,industrial_use_type_Metallurgy,industrial_use_type_Non-industrial,industrial_use_type_Service/Tourism,govermental_use_type_Govermental Buildings,govermental_use_type_Non-govermental,govermental_use_type_Police Offices,flexible_superstructure_available,flexible_superstructure_unavailable
21,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
27,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
43,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
46,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
92,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519918,1,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
519956,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
519969,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
519980,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1


In [64]:
for i in obj_cols[2:5]:
    tmp_train.groupby(['technical_solution_proposed', i])[i].count()

In [65]:
def freq_encoding(data, cat_feat):
  grouped_data = data.groupby([cat_feat]).size()/data.shape[0]
  data.loc[:, f'{cat_feat}_encode'] = data[cat_feat].map(grouped_data)
  return data

In [66]:
for col in ['land_surface_condition',
 'type_of_foundation',
 'type_of_roof',
 'type_of_ground_floor',
 'type_of_other_floor',
 'position',
 'building_plan_configuration']:
  tmp_train = freq_encoding(tmp_train, col)

In [67]:
cols_tmp = [i for i in tmp_train if i not in obj_cols and i != 'damage_grade']


In [68]:
train_final = pd.concat([tmp_var_ohe,tmp_train[cols_tmp]],axis=1)

In [69]:
y = tmp_train['damage_grade']

In [70]:
cross_validation(train_final,y,HistGradientBoostingClassifier(random_state=0))

here is the : 0.6492853523506987
              precision    recall  f1-score   support

         1.0       0.88      0.68      0.77       969
         2.0       0.67      0.79      0.73      1063
         3.0       0.80      0.76      0.78      1689
         4.0       0.50      0.16      0.24      2247
         5.0       0.61      0.91      0.73      3393

    accuracy                           0.66      9361
   macro avg       0.69      0.66      0.65      9361
weighted avg       0.65      0.66      0.63      9361

here is the : 0.655392058885115
              precision    recall  f1-score   support

         1.0       0.89      0.73      0.80       968
         2.0       0.69      0.77      0.73      1063
         3.0       0.78      0.74      0.76      1689
         4.0       0.50      0.17      0.25      2247
         5.0       0.62      0.92      0.74      3393

    accuracy                           0.67      9360
   macro avg       0.70      0.66      0.66      9360
weighted avg

([0.6492853523506987,
  0.655392058885115,
  0.6551616343093557,
  0.6565556034966008,
  0.6633030024572453],
 [])

t

In [74]:
tmp = pd.concat([train_final,y],axis=1)

In [84]:
df_corr = pd.DataFrame(abs(tmp.corr()))

/var/folders/yt/n9_2kr5s6nx_5b2y_kx3mnnm0000gn/T/ipykernel_39890/726426553.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_corr = pd.DataFrame(abs(tmp.corr()))


In [101]:
cc = df_corr['damage_grade'].sort_values(ascending=False).head(-1).index
dipake = cc[1:]
cross_validation(train_final[dipake],y,svm.SVC(random_state=0))